In [1]:
from elftools.elf.elffile import ELFFile
import frida
import sys
from time import sleep

In [2]:
def generate_function_list(binary):
    """
    Generate a list of all the functions defined by the target executable.
    """
    functions = []
    with open(binary, "rb") as f:
        elf = ELFFile(f)
        for section in elf.iter_sections():
            if section.header.sh_type == 'SHT_SYMTAB':
                for symbol in section.iter_symbols():
                    if symbol.entry.st_info.type == 'STT_FUNC':
                        functions.append(symbol.name)
    return functions

In [3]:
binary_path = "./test/test_char"
args = ["arg1", "arg2"]

function_list = generate_function_list(binary_path)
print(function_list)

['deregister_tm_clones', 'register_tm_clones', '__do_global_dtors_aux', 'frame_dummy', '__libc_start_main@GLIBC_2.34', '_fini', 'strlen@GLIBC_2.2.5', 'f', 'printf@GLIBC_2.2.5', '_start', 'g', 'main', '__cxa_finalize@GLIBC_2.2.5', '_init']


In [9]:
def make_script(address, n):
    args_str = ', '.join(f'args[{i}]' for i in range(n))
    return """
            console.log('Attaching interceptor to address: 0x%s');
            Interceptor.attach(ptr('%s'), {
                onEnter: function (args) {
                    console.log('Interceptor entered for address: 0x%s');
                    send({ function: '%s', args: [%s] });
                }
            });
        """ % (address, address, address, address, args_str)

In [10]:
entries = []

def on_message(message, data):
    print(message)
    if message["type"] == "send" and message["payload"] != "done":
        function_name = message["payload"]["function"]
        try:
            function_args = message["payload"]["args"]
            io="input"
        except:
            function_args=message["payload"]["ret"]
            io="output"
        entries.append((function_name,function_args))

# Run the binary
process = frida.spawn(binary_path, argv=[binary_path] + args)

sleep(1)

session = frida.attach(process)

function_list=[4198496, 4198512, 4198528, 4198761, 4198825, 4198864]
n=[1, 1, 0, 1, 1, 2]

script_txt=""
for f,i in zip(function_list,n):
    script_txt+= make_script(f,i)
    script_txt+="\n"
    
script = session.create_script(script_txt)
script.on("message",on_message)
script.load()

frida.resume(process)

    # Wait for the script to complete
    #script.join()
sleep(5)

Attaching interceptor to address: 0x4198496
{'type': 'error', 'description': 'Error: access violation accessing 0x401060', 'stack': 'Error: access violation accessing 0x401060\n    at value (frida/runtime/core.js:367)\n    at <eval> (/script1.js:8)', 'fileName': 'frida/runtime/core.js', 'lineNumber': 367, 'columnNumber': 1}
Result: 6
